In [1]:
import pandas as pd
import os
import ast

labeled_data_path = os.getcwd()+'\\Label'
test_dataset_path = os.getcwd()+'\\Test'

geo_trace_pts = {} #전체 중복 경로 횟수

def jaro_similarity(list1, list2):
    # 두 리스트의 길이를 구함
    len_list1 = len(list1)
    len_list2 = len(list2)

    # 두 리스트 중에서 더 긴 길이를 기준으로 함
    max_len = max(len_list1, len_list2)

    # 일치하는 값의 개수를 세기 위한 변수 초기화
    match_count = 0

    # 일치하는 값의 인덱스를 저장하기 위한 리스트 초기화
    matches_list1 = [False] * len_list1
    matches_list2 = [False] * len_list2

    # 첫 번째 단계: 일치하는 값 세기
    for i in range(len_list1):
        start = max(0, i - int(max_len / 2))
        end = min(i + int(max_len / 2) + 1, len_list2)

        for j in range(start, end):
            if not matches_list2[j] and list1[i] == list2[j]:
                matches_list1[i] = True
                matches_list2[j] = True
                match_count += 1
                break

    # 일치하는 값이 하나도 없으면 0 반환
    if match_count == 0:
        return 0.0

    # 두 번째 단계: 일치하는 값의 순서에 따라 가중치 적용
    transpositions = 0
    k = 0
    for i in range(len_list1):
        if matches_list1[i]:
            while not matches_list2[k]:
                k += 1

            if list1[i] != list2[k]:
                transpositions += 1

            k += 1

    # 유사도 계산
    jaro_similarity = ((match_count / len_list1 + match_count / len_list2 + (match_count - transpositions / 2) / match_count)/ 3.0)

    return jaro_similarity

def com_shell(label):
    
    if label in shell:
        print('정상 경로')
        return True
    else:
        print('비정상 경로')
        return False

# Create a function to get the grid label of the coordinate point
def find_label_for_point(lat, lng, grid_info):
    for index, row in grid_info.iterrows():
        # Convert string representations to tuples
        min_lat, min_lng = ast.literal_eval(row['Min Latitude, Min Longitude'])
        max_lat, max_lng = ast.literal_eval(row['Max Latitude, Max Longitude'])

        if min_lat <= lat <= max_lat and min_lng <= lng <= max_lng:
            return row['Grid Name']

    return None

# 디렉토리 내의 모든 CSV 파일 가져오기
csv_files = [f for f in os.listdir(labeled_data_path) if f.endswith('.csv')]

# 각 CSV 파일을 다른 데이터프레임에 저장하기 위한 딕셔너리 생성
dfs = {}

# 각 CSV 파일을 읽어서 데이터프레임에 저장
for file in csv_files:
    # CSV 파일의 경로
    file_path = os.path.join(labeled_data_path, file)
    
    # 파일명을 key로 사용하여 데이터프레임 생성
    df_key = os.path.splitext(file)[0]  # 파일명에서 확장자 제거
    dfs[df_key] = pd.read_csv(file_path)['grid_label']
     
    gridDataLabel = 0 # grid lable 중복 제거 변수
    geo_trace = [] #geo trace 리스트
    
    for grid_label in dfs[df_key]:
        if gridDataLabel != grid_label:
            gridDataLabel = grid_label
            geo_trace.append(grid_label)

    if geo_trace == []:
        geo_trace.append(grid_label)

    tuple_geo = tuple(geo_trace)
    geo_trace_pts[tuple_geo] = geo_trace_pts.get(tuple_geo, 0) + 1
    sorted_geo_trace_pts = dict(sorted(geo_trace_pts.items(), key=lambda x: x[1], reverse=True))

    
# 유사도 비교
shell_list = []
pair_list = []
pair_dict = {}
flatten_list = []
flatten_dict = {}

for key1, value1 in sorted_geo_trace_pts.items():
    list1 = list(key1)
    for key2, value2 in sorted_geo_trace_pts.items():
        if key1 == key2:
            continue
        list2 = list(key2)
        if jaro_similarity(list1, list2)>0.80:
            jaro_sim = jaro_similarity(list1, list2)
            shell_list.extend(list2)

shell = set(shell_list)

# print(listpath)
# Read the test CSV file

grid_info_file = 'grid_information_with_paths.csv'
grid_info = pd.read_csv(grid_info_file)
listpath = []
current_label = 'A'

for filename in os.listdir(test_dataset_path):
    if filename.endswith('.csv'):
        csv_file_path = os.path.join(test_dataset_path, filename)
        df = pd.read_csv(csv_file_path)
        previous_label = ""
        
        for index, row in df.iterrows():
            lat = row['lat']
            lng = row['lng']
            
            label = find_label_for_point(lat, lng, grid_info)
            
            if current_label != label:
                current_label = label
                print("사용자 현재 위치", label)
                com_shell(label)
                listpath.append(label)

사용자 현재 위치 SGDABC
정상 경로
사용자 현재 위치 SGDAAD
정상 경로
사용자 현재 위치 SGDABC
정상 경로
사용자 현재 위치 SGDADD
정상 경로
사용자 현재 위치 SGDADC
정상 경로
사용자 현재 위치 SGDCAB
정상 경로
사용자 현재 위치 SGDACB
정상 경로
사용자 현재 위치 SGDACA
정상 경로
사용자 현재 위치 SGCBDB
정상 경로
사용자 현재 위치 SGCBBD
정상 경로
사용자 현재 위치 SGDAAC
정상 경로
사용자 현재 위치 SGDAAD
정상 경로
사용자 현재 위치 SGDABC
정상 경로
사용자 현재 위치 SGDAAD
정상 경로
사용자 현재 위치 SGDACB
정상 경로
사용자 현재 위치 SGDACA
정상 경로
사용자 현재 위치 SGDACC
정상 경로
사용자 현재 위치 SGCBDD
비정상 경로
사용자 현재 위치 SGDACC
정상 경로
사용자 현재 위치 SGDCAA
비정상 경로
사용자 현재 위치 SGDCCC
정상 경로
사용자 현재 위치 RGBAAA
정상 경로
사용자 현재 위치 RGABBB
비정상 경로
사용자 현재 위치 RGABBD
비정상 경로
사용자 현재 위치 RGABDB
비정상 경로
사용자 현재 위치 RGBAC
비정상 경로
사용자 현재 위치 RGBCAB
비정상 경로
사용자 현재 위치 RGBCBA
비정상 경로
사용자 현재 위치 RGBCBC
비정상 경로
사용자 현재 위치 RGBCBD
비정상 경로
사용자 현재 위치 RGBDAC
비정상 경로
사용자 현재 위치 RGBDAA
비정상 경로
사용자 현재 위치 RGBDAB
비정상 경로
사용자 현재 위치 RGBBCD
비정상 경로
사용자 현재 위치 RGBBDC
비정상 경로
사용자 현재 위치 RGBBDA
비정상 경로
사용자 현재 위치 RGBBBC
비정상 경로
사용자 현재 위치 RGBBBA
비정상 경로
사용자 현재 위치 RGBBAB
비정상 경로
사용자 현재 위치 SGDDCD
비정상 경로
사용자 현재 위치 SGDDDC
비정상 경로
사용자 현재 위치 SGDDCD
비정상 경로
사용자 현재 위치 SGD

In [3]:
import os
import string
import datetime

import folium
import pandas as pd
import geopy.distance

# Paths of all the datasets
train_dataset_path = os.getcwd()+'\\Train'
test_dataset_path = os.getcwd()+'\\Test'
labeled_data_path = os.getcwd()+'\\Label'

# Define a function to convert numbers into corresponding letter labels
def num_to_letter(num):
    '''
    num         : number that we have to convert
    '''
    return string.ascii_uppercase[num]

# Define a function to check whether the path passes through the grid
def is_path_in_grid(south, west, north, east, path_points):
    '''
    south       : minimum latitude
    west        : minimum longitude
    north       : maximum latitude
    east        : maximum longitude
    path_points : coordinate points
    '''
    for lat, lng in path_points:
        if south <= lat <= north and west <= lng <= east:
            return True
    return False

# Create a function to get the grid label of the coordinate point
def get_grid_label(lat, lng, final_grids):
    '''
    lat         : latitude
    lng         : longitude
    final_grids : all cells and their minimum/maximum latitude/longitude
    '''
    for south, west, north, east, grid_label in final_grids:
        if south <= lat <= north and west <= lng <= east:
            return grid_label
    return None

# Approximate border coordinates of South Korea
south_korea_bounds = [34, 125.5, 39, 130]
 
# Create a map 
m = folium.Map(location=[(south_korea_bounds[0] + south_korea_bounds[2]) / 2,
                         (south_korea_bounds[1] + south_korea_bounds[3]) / 2],
               zoom_start=7)

# Read waypoint
path_points = []
directory = train_dataset_path
path_dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        data = pd.read_csv(file_path, encoding='utf-8')
        path_dataframes.append(data)
        points = data[['lat', 'lng']].values.tolist()
        path_points.extend(points)
 
        # draw a line on the map
        folium.PolyLine(points, color='red', weight=2.5, opacity=1).add_to(m)

# Initialize grid queue
grid_queue = []
final_grids = []  # Used to store the final small grid
initial_lat_step = (south_korea_bounds[2] - south_korea_bounds[0]) / 26
initial_lon_step = (south_korea_bounds[3] - south_korea_bounds[1]) / 26
#initial_lat_step *= 0.94
#initial_lon_step *= 0.94
 
for i in range(26):
    for j in range(26):
        south = south_korea_bounds[0] + i * initial_lat_step
        north = south_korea_bounds[0] + (i + 1) * initial_lat_step
        west = south_korea_bounds[1] + j * initial_lon_step
        east = south_korea_bounds[1] + (j + 1) * initial_lon_step
        grid_queue.append((south, west, north, east, num_to_letter(i) + num_to_letter(j)))

# Process grid queue
min_size_km = 1  # Minimum grid size (km)
subdivisions = ['A', 'B', 'C', 'D']  # Split label
while grid_queue:
    south, west, north, east, grid_label = grid_queue.pop(0)
    grid_size_km = min(geopy.distance.distance((south, west), (south, east)).km,
                       geopy.distance.distance((south, west), (north, west)).km)
 
    if grid_size_km > min_size_km and is_path_in_grid(south, west, north, east, path_points):
        mid_lat = (south + north) / 2
        mid_lon = (west + east) / 2
        grid_queue.append((south, west, mid_lat, mid_lon, grid_label + 'C'))
        grid_queue.append((mid_lat, west, north, mid_lon, grid_label + 'A'))
        grid_queue.append((south, mid_lon, mid_lat, east, grid_label + 'D'))
        grid_queue.append((mid_lat, mid_lon, north, east, grid_label + 'B'))
    else:
        final_grids.append((south, west, north, east, grid_label))
        folium.Rectangle(
            bounds=[[south, west], [north, east]],
            color='#0000FF',
            fill=True,
            fill_opacity=0.1
        ).add_to(m)
        # Add a green label to the center of the grid
        folium.Marker(
            location=[(south + north) / 2, (west + east) / 2],
            icon=folium.DivIcon(html=f'<div style="font-size: 8pt; color: yellow;">{grid_label}</div>')
        ).add_to(m)

grid_data = {
    'Grid Name': [label for _, _, _, _, label in final_grids],
    'Min Latitude, Min Longitude': [(south, west) for south, west, _, _, _ in final_grids],
    'Max Latitude, Max Longitude': [(north, east) for _, _, north, east, _ in final_grids]
}
grid_df = pd.DataFrame(grid_data)

# Save the grid information to a CSV file
grid_df.to_csv('grid_information_with_paths.csv', index=False)

# Assign final grid labels to waypoints
for data in path_dataframes:
    data['grid_label'] = data.apply(lambda row: get_grid_label(row['lat'], row['lng'], final_grids), axis=1)

# checking if the directory demo_folder exist or not. 
if not os.path.exists(labeled_data_path):  
    os.makedirs(labeled_data_path)

# Save the updated DataFrame to a new CSV file
for idx, df in enumerate(path_dataframes):
    labeled_file = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
    df.to_csv(f'Label/{labeled_file}.csv', index=False)

# Save or show map
m.save('south_korea_grid_map_with_subdivided_paths.html')